# Закат Трансформеров


### Языковое моделирование

- Учимся предсказывать следующий токен
- Токен, часть слова
- $(u_1 , …, u_L)$ — последовательность входов
- $(y_1 , …, y_L )$ — последовательность выходов
- $(y_1 , …, y_L-1 ) = (u_2 , …, u_L )$
- Мера успешности — перплексия
- Сравнение перплексий имеет смысл только с одинаковыми разбиениями на токены

![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/Modelling.png)

### Long Range Arena
- Статья: LRA: A Benchmark for Eﬃcient Transformers
- Arxiv: 2011.04006
- Организация: Google Research
- Основная идея: новый бенчмарк для трансформеров на длинных последовательностях
- Репозиторий: google-research/long-range-arena
- Особенности:
- Никаких предобученных моделей
- Ориентированность на вариации трансформеров
<div>
<img src="https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/Bench.png" width="500"/>
</div>




### Long Range Arena задачи

- **Long ListOps**: парсинг математических выражений
    - Вход: [max 4 3 [min 2 3 ] 1 0 [median 1 5 8 9 2]]
    - Выход: 5
    - Контекст: 2000
<div>
<img src="https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/Math.png" width="200"/>
</div>   
 
- **Text**: классификация текстов на уровне байтов
    - Отзывы с IMDB, анализ тональности
    - Контекст: 4000
- **Retrieval**: сравнение документов на уровне байтов
    - Предсказание цитирования одной научной статьи другой
    - Статьи из ACL Anthology Network
    - Две “башни”, конкатенация выходов, линейный слой
    - Контекст: 4000 для каждой статьи    
- **Image**: классификация картинок
    - Картинка как последовательность пикселей
    - CIFAR-10, 32x32, в каждом пикселе 256 значений
    - 10 классов
- **Pathﬁnder**: поиск пути на картинке
    - Бинарная классификация: есть ли путь?
    - Картинки тоже 32x32
- **Path-X**: поиск пути на большой картинке
    - Картинки 128x128
    - Контекст: 16000
    


<div>
<img src="https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/Path.png" width="200"/>
</div>


### Трансформеры
- Квадратичная сложность по времени и памяти,потому что $QK^T$
- Линейная сложность по времени на каждом шаге инференса
- Параллельное обучение: один forward для всей последовательности, всегда teacher forcing, маски для будущих токенов

![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/Transformers.png)

![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/Bench1.png)

![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/BenchS4.png)

![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/S4_1.png)

# РНС

- Константная сложность по времени на каждом шаге инференса
- Последовательное обучение, шаг за шагом

 $$ x_ {k}  = \sigma (  Ax_ {k-1}  +  Bu_ {k})$$
 $$ y_ {k}  = Cx_ {k} $$ 

![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/RES_1.png)

### Линейные РНС **Которые на самом деле свёрточные сети**
- Константная сложность по времени на каждом шаге инференса
- Параллельное обучение через свёртки
- Свёртки через БПФ работают за **O(L log(L))**
- Плохое качество при наивном использовании


$$ x_k = \sum_{j=0}^{k-1} A^{j}Bu_{k-j}$$


$$ \mathcal{K} = (CB, CAB, \ldots, CA^{L-1}B)$$


$$y = \text{conv1d}(\mathcal{K}_L \ldots \mathcal{K}_1, u_1 \ldots u_L)$$

![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/RNC.png)

### Stack more layers!
- 6 слоёв в целом аналогичных трансформерным
- Нелинейность в MLP
- Остаточные связи
- Layer/batch normalization

![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/RNC_2.png)
![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/rnc_3.png)
![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/rnc_4.png)

## Модели на линейных РНС
- LRU, Linear Recurrent Units
- SSM, модели в пространстве состояний, такие как S4 и H3
- RWKV, Receptance Weighted Key Value
- RetNet, Retentive Network

### LRU: Linear Recurrent Units
- Статья: Resurrecting Recurrent Neural Networks for Long Sequences
- Arxiv: 2303.06349
- Организация: DeepMind
- Основная идея: объяснение, что нужно сделать с линейными РНС, чтобы они начали нормально работать
- Особенности:
- Статья вышла гораздо позже S4, но просто объясняет, почему оно работает
- Начинаем с тривиальной модели, навешиваем разные трюки
- Модель та, которая была описана раньше: линейные РНС, чередующиеся с нелинейными многослойными перцептронами

### LRU, трюк 1: диагонализация A
- Сложно возводить в степень плотную матрицу A
- Гораздо проще для диагональной матрицы!
- Стабильность: $|𝜆_i | < 1$ (чтобы градиенты не взрывались)
- Эквивалентная инициализация: равномерно на единичном круге в ℂ


$$\textbf{Модель}$$
$$x_k = Ax_{k-1} + Bu_k$$
$$y_k = Cx_k$$

$$\textbf{Спектральное разложение}$$
$$x_k = \sum_{j=0}^{k-1} A^j Bu_{k-j}$$
$$A = P \Lambda P^{-1}, \quad P \in \mathbb{C}^{N \times N}$$
$$\Lambda = \text{diag}(\lambda_1, \ldots, \lambda_N) \in \mathbb{C}^{N \times N}$$
$$\vec{x}_k := P^{-1} x_k, \quad \vec{B} := P^{-1}B$$
$$\vec{x}_k = \sum_{j=0}^{k-1} \Lambda^j \vec{B} u_{k-j}$$
![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/LRU_1.png)
![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/LRU_2.png)

### LRU, трюк 2: полярная система координат
- Переходим к полярной системе координат
- То есть разделяем амплитуду и фазу
- Амплитуда: exp(-𝞶)
- Влияет на оптимизацию: Adam’у проще

$$\text{Полярная система координат}$$

$$\Lambda = \text{diag}(\alpha + i\beta), \quad \alpha \in \mathbb{R}^N, \beta \in \mathbb{R}^N$$

$$\downarrow$$

$$\Lambda = \text{diag}(e^{-\nu+i\theta}), \quad \nu \in \mathbb{R}^N, \theta \in \mathbb{R}^N$$
![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/LRU_4.png)
![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/LRU_3.png)

## LRU, трюк 3: стабильная параметризация

- Как выглядит $|𝜆_i | < 1$ в полярной системе?
- $|𝜆_i | < 1$ означает амплитуду < 1, то есть 𝞶 > 0
- Добавляем нелинейность, гарантирующую 𝞶 > 0
- Ещё одну экспоненту: чтобы было очень много значений около 0

$$\text{Полярная система координат}$$

$$\Lambda = \text{diag}(e^{-\nu+i\theta}), \quad \nu \in \mathbb{R}^N, \theta \in \mathbb{R}^N$$

$$\downarrow$$

$$\Lambda = \text{diag}(e^{e^{-\nu}+i\theta}), \quad \nu \in \mathbb{R}^N, \theta \in \mathbb{R}^N$$


![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/LRU_5.png)

## LRU, трюк 4: инициализация на диске вместо круга

- Заводим минимальный и максимальный радиусы
- Чем ближе к $r_{max}$ к 1, тем более нестабильна модель
- Но чем ближе к $r_{max}$ к 1, тем лучше модель учит длинные последовательности
- Поэтому $r_{max}$ = 0.99

![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/LRU_6.png)
![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/LRU_7.png)

## LRU, трюк 5: нормализация входов

- Параметр 𝛾 адаптивно масштабирует входы под собственные вектора
    $$x_k = \Lambda x_{k-1} + e^{\gamma} \circ (B u_k), \gamma \in \mathbb{R}^N$$


$$\text{Init: } \gamma_i \leftarrow \log\sqrt{1 - |\lambda_i|^2}$$
![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/LRU_8.png)

## LRU: итоговая картина

![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/LRU_9.png)

### S4
- Статья: Eﬃciently Modeling Long Sequences with Structured State Spaces
- Arxiv: 2111.00396
- Организация: Stanford University
- Репозиторий: HazyResearch/state-spaces
- Основная идея: демонстрация прорыва на LRA при использовании линейных РНС с особой параметризацией

![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/S4_2.png)

### Пространства состояний
Линейная независимая от времени непрерывная система:
- u(t) - входной непрерывный сигнал,
- x(t) - скрытое состояние системы,
- y(t) - выходной непрерывный сигнал


$$ \dot{x}(t) = Ax(t) + Bu(t) \\
y(t) = Cx(t) + Du(t)$$ 
Пример из механики: x(t) как конкатенация векторов координат и скоростей (да-да, 6N-мерное пространство)

![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/S4_3.png)

### Пространства состояний: дискретизация

- D = 0, убираям прямую связь, у нас для этого есть остаточные связи в сети
- Дискретизуем с шагом Δ, обучаемый параметр
- Используем билинейное преобразование
- Теперь Δ влияет на параметризацию новых A и B
- Дискретизация неявно задаёт нормализацию и инициализацию

$$\vec{A} = (I - \Delta/2 \cdot A)^{-1}(I + \Delta/2 \cdot A)$$

$$\vec{B} = (I - \Delta/2 \cdot A)^{-1}\Delta B$$

$$\vec{C} = C$$

$$x_k = \vec{A}x_{k-1} + \vec{B}u_k$$

$$y_k = \vec{C}x_k$$

![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/S4_4.png)

### S4, трюк 1: инициализация через HiPPO

- Статья: HiPPO: Recurrent Memory with Optimal Polynomial Projections
- Arxiv: 2008.07669
- Организация: Stanford University
- Основные выводы:
    - Используем фиксированную матрицу A
    - Она заставляет систему аппроксимировать историю входов в скрытом состоянии
    - Аппроксимация через коэффициенты многочленов Лежандра


$$A_{nk} = \begin{cases}
(2n + 1)^{1/2}(2k + 1)^{1/2} & \text{if } n > k \\
n + 1 & \text{if } n = k \\
0 & \text{if } n < k
\end{cases}$$
![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/s4_6.png)
![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/s4_5.png)


### S4, трюк 2: diagonal plus low-rank
- Всё та же диагонализация
![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/S4_7.png)

### S4, Результаты
![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/S4_R_1.png)
![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/S4_R_2.png)

### RWKV
- Статья: RWKV: Reinventing RNNs for the Transformer Era
- Arxiv: 2305.13048
- Организация: EleutherAI и другие
- Репозиторий: BlinkDL/RWKV-LM
- Основная идея: наполовину транформер, наполовину РНС с 14 миллиардами параметров и поддержкой в HF Transformers

![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/rwkv_1.png)

#### RWKV: wkv и time-mixing
- Выкидываем матрицу Q
- w: не зависящий от текущего токена вектор весов, определяющий затухание
- Если $e^{-w}$ близко к 0, концентрируемся на близких токенах, если к 1, учитываем глобальный контекст
- Receptance, σ(r): гейт для удаления лишней информации из истории
- Интерполяция входов

**Обычное внимание**

$$\text{Attn}(Q, K, V) = \text{softmax}(QK^T)V$$

$$\text{Attn}(Q, K, V)_t = \frac{\sum_{i=1}^t e^{\mathbf{q}_t^\mathsf{T}\mathbf{k}_i}\mathbf{v}_i}{\sum_{i=1}^t e^{\mathbf{q}_t^\mathsf{T}\mathbf{k}_i}}$$

**Псевдо-внимание wkv**
$$wkv_t = \frac{\sum_{i=1}^{t-1}e^{-(t-i-1)w+\mathbf{k}_i} \mathbf{v}_i + e^{\mathbf{u}+\mathbf{k}_t}\mathbf{v}_t}{\sum_{i=1}^{t-1}e^{-(t-i-1)w+\mathbf{k}_i} + e^{\mathbf{u}+\mathbf{k}_t}}$$

$$k_t = W_k (\mu_k\mathbf{x}_t + (1 - \mu_k)\mathbf{x}_{t-1})$$

$$v_t = W_v (\mu_v\mathbf{x}_t + (1 - \mu_v)\mathbf{x}_{t-1})$$

$$r_t = W_r (\mu_r\mathbf{x}_t + (1 - \mu_r)\mathbf{x}_{t-1})$$

$$o_t = W_o \cdot (\sigma(r_t) \odot wkv_t)$$

![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/rwkv_3.png)

#### RWKV: wkv как рекуррентная сеть
- На самом деле это две рекуррентных сети

$$wkv_t = \frac{\sum\limits_{i=1}^{t-1} e^{-(t-i-1)w+k_i}v_i + e^{u+k_t}v_t}{\sum\limits_{i=1}^{t-1} e^{-(t-i-1)w+k_i} + e^{u+k_t}},$$

$$a_0, b_0 = 0,$$

$$wkv_t = \frac{a_{t-1} + e^{u+k_t}v_t}{b_{t-1} + e^{u+k_t}},$$

$$a_t = e^{-w}a_{t-1} + e^{k_t}v_t,$$

$$b_t = e^{-w}b_{t-1} + e^{k_t}.$$
![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/rwkv_2.png)

### RWKV: полная модель
- R: вектора “принятия” информации из прошлого
- W: вектора с весами позиционного затухания
- K: аналог ключей внимания
- V: аналог значений внимания
- Вся сеть: слои из комбинаций time-mixing и channel-mixing блоков
- Вместо свёрток в обучении — аналог внимания, wkv
- wkv работает за линейное время, нет аналога $QK^{T}$

![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/rwkv_4.png)

## Channel Mixing
Слой смешивания каналов принимает на вход $x$, соответствующий данному токену, и $x$, соответствующий предыдущему токену, который мы называем $last_x$. $last_x$ был сохранен в состоянии этого слоя RWKV. Остальные входы - это выученные параметры RWKV. Сначала мы линейно интерполируем $x$ и $last_x$, используя выученные веса. Этот интерполированный $x$ мы подаем на вход двухслойной сети с квадратичной активацией $relu$ и, наконец, умножаем на сигмоидную активацию другой сети с прямой передачей (в классических терминах RNN это называется gating). Обратите внимание, что с точки зрения использования памяти матрицы $W_k,W_r,W_v$ содержат почти все параметры (самая маленькая из них 1024x1024 - матрица, а остальные переменные - просто 1024-мерные векторы). А матричные умножения (@ в python) вносят подавляющее большинство необходимых вычислений.

In [5]:
import numpy as np
from torch import load as torch_load  # Only for loading the model weights
from tokenizers import Tokenizer

layer_norm = lambda x, w, b : (x - np.mean(x)) / np.std(x) * w + b
exp = np.exp
sigmoid = lambda x : 1/(1 + exp(-x))

In [ ]:
def channel_mixing(x, last_x, mix_k, mix_r, Wk, Wr, Wv):
    k = Wk @ ( x * mix_k + last_x * (1 - mix_k) )
    r = Wr @ ( x * mix_r + last_x * (1 - mix_r) )
    vk = Wv @ np.maximum(k, 0)**2
    return sigmoid(r) * vk, x

# Time Mixing

Временное смешение начинается так же, как и смешение каналов, с интерполяции $x$ этого токена с x последнего токена. Затем мы применяем выученные 1024x1024 матрицы, чтобы получить векторы $"ключ"$, $"значение"$ и $"рецепция"$. В следующей части происходит волшебство.

In [4]:
def time_mixing(x, last_x, last_num, last_den, decay, bonus, mix_k, mix_v, mix_r, Wk, Wv, Wr, Wout):
    k = Wk @ ( x * mix_k + last_x * (1 - mix_k) )
    v = Wv @ ( x * mix_v + last_x * (1 - mix_v) )
    r = Wr @ ( x * mix_r + last_x * (1 - mix_r) )

    wkv = (last_num + exp(bonus + k) * v) /      \
          (last_den + exp(bonus + k))
    rwkv = sigmoid(r) * wkv

    num = exp(-exp(decay)) * last_num + exp(k) * v
    den = exp(-exp(decay)) * last_den + exp(k)

    return Wout @ rwkv, (x,num,den)

# Внимание RWKV"
Прежде чем перейти к сути механизма, сделаем замечание, что, хотя переменные, поступающие в механизм внимания, являются 1024-мерными (мы говорим, что у них 1024 канала), все каналы вычисляются независимо друг от друга. Поэтому мы просто рассмотрим, что происходит с одним каналом, рассматривая переменные как скаляры. 

Теперь давайте рассмотрим переменную $num$. Чтобы сделать математические обозначения чище, давайте переименуем $num$ и $den$ в $\alpha$ and $\beta$. Оба $\alpha$ and $\beta$ хранятся в состоянии RWKV. Для каждого нового токена, $\alpha$ вычисляется как $\alpha_i = e^{-w} \alpha_{i-1} +e^{k_i} v_i$, где $i$ - индекс текущего токена. Мы определили $w = exp(decay)$, заметим, что $w$ всегда положительно. 

По индукции имеем $\alpha_i = \sum_{j=1}^i e^{-(i-j)w+k_j} v_j$. Аналогично, $\beta_i = \sum_{j=1}^i e^{-(i-j)w+k_j}$ . Обратите внимание, что $\alpha_i$ выглядит как взвешенная сумма $v_j$, в то время как $\beta_i$ это просто сумма весов. Таким образом, $\frac{\alpha_i}{\beta_i}$ становится средневзвешенным значением из  $v_j$. Подставляя формулы для $\alpha_{i-1}$ и $\beta_{i-1}$в определение $wkv$ и обозначая $bonus$ через $u$, получаем
$$\text{wkv}_i = \frac{ \sum_{j=1}^{i-1} e^{-(i-1-j)w+k_j} v_j + e^{u+k_i} v_i }{\sum_{j=1}^{i-1} e^{-(i-1-j)w+k_j} + e^{u+k_i}}.$$

Это $wkv$ средневзвешенное значение $v$ с весами $k$ в соответствии с , но при этом текущий $v_i$ имеет бонусный ($u$) дополнительный вес, а предыдущие $v_j$ имеют геометрически меньшие веса, чем дальше они находятся.

Для справки, стандартный трансформер внимания принимает векторы "запрос", "ключ" и "значение" и  $q,k,v$  выдает $$$\frac{\sum_{j=1}^i e^{q_i^\top k_j} v_j}{\sum_{j=1}^i e^{q_i^\top k_j}}.$$

После вычисления $wkv$, time mixing умножается на "приемник(receptance)" $sigmoid(r)$. Перед тем как вернуть результат, он выполняет окончательное линейное преобразование.

### Преобразование в выходные вероятности
После прохождения 24 слоев временного смешивания и смешивания каналов нам нужно преобразовать конечный выход в предсказанные вероятности для следующего токена.
```
x = layer_norm(x, *params('ln_out')) 
x = params('head')[0] @ x 
e_x = exp(x-np.max(x)) 
probs = e_x / e_x.sum() # Softmax of x 
```
Сначала мы делаем нормализацию слоев. Затем мы умножаем на матрицу размерностью 50277x1024 params('head')[0], заданную параметрами RWKV, получая 50277-мерный вектор. Чтобы получить распределение вероятностей по токенам (то есть 50277-мерный неотрицательный вектор, который в сумме равен 1), мы прогоняем наш $x$ через функцию "softmax". Softmax для $x$ - это просто $exp(x)/sum(exp(x))$. Однако вычисление $exp(x)$ может привести к численным переполнениям, поэтому мы вычисляем эквивалентную функцию $exp(x-max(x))/sum(exp(x-max(x)))$

In [7]:
def RWKV(model, token, state):
    params = lambda prefix : [model[key] for key in model.keys() if key.startswith(prefix)]

    x = params('emb')[0][token]
    x = layer_norm(x, *params('blocks.0.ln0'))

    for i in range(N_LAYER):
        x_ = layer_norm(x, *params(f'blocks.{i}.ln1'))
        dx, state[i][:3] = time_mixing(x_, *state[i][:3], *params(f'blocks.{i}.att'))
        x = x + dx

        x_ = layer_norm(x, *params(f'blocks.{i}.ln2'))
        dx, state[i][3] = channel_mixing(x_, state[i][3], *params(f'blocks.{i}.ffn'))
        x = x + dx

    x = layer_norm(x, *params('ln_out'))
    x = params('head')[0] @ x

    e_x = exp(x-np.max(x))
    probs = e_x / e_x.sum() # Softmax of x

    return probs, state


In [6]:
def sample_probs(probs, temperature=1.0, top_p=0.85):
    sorted_probs = np.sort(probs)[::-1]
    cumulative_probs = np.cumsum(sorted_probs)
    cutoff = sorted_probs[np.argmax(cumulative_probs > top_p)]
    probs[probs < cutoff] = 0
    probs = probs**(1/temperature)
    return np.random.choice(a=len(probs), p=probs/np.sum(probs))

# Available at https://huggingface.co/BlinkDL/rwkv-4-pile-430m/resolve/main/RWKV-4-Pile-430M-20220808-8066.pth
MODEL_FILE = './data_z/RWKV-4-Pile-430M-20220808-8066.pth'
N_LAYER = 24
N_EMBD = 1024

print(f'\nLoading {MODEL_FILE}')
weights = torch_load(MODEL_FILE, map_location='cpu')
for k in weights.keys():
    if '.time_' in k: weights[k] = weights[k].squeeze()
    weights[k] = weights[k].float().numpy() # convert to f32 type


# Available at https://github.com/BlinkDL/ChatRWKV/blob/main/20B_tokenizer.json
tokenizer = Tokenizer.from_file("./data_z/20B_tokenizer_1.json")

print(f'\nPreprocessing context')

context = "\nIn a shocking finding, scientist discovered a herd of dragons living in a remote, previously unexplored valley, in Tibet. Even more surprising to the researchers was the fact that the dragons spoke perfect Chinese."



Loading ./data_z/RWKV-4-Pile-430M-20220808-8066.pth

Preprocessing context


Код использует RWKV для продолжения следующего текста:

"В результате шокирующего открытия ученые обнаружили стадо драконов, живущих в отдаленной, ранее не исследованной долине в Тибете. Еще более удивительным для исследователей оказался тот факт, что драконы прекрасно говорили по-китайски".

Сначала нам нужно преобразовать этот текст в ряд лексем (числа от 0 до 50276, обозначающие слова/символы/лексики в нашем словаре). 
Делаем это с помощью внешней библиотеки tokenizer.encode(context).ids.

Далее нам нужно обработать эту последовательность токенов в состояние RWKV. По сути, RWKV представляет собой функцию, которая принимает токен и состояние и выдает распределение вероятности для следующего токена и нового состояния. Конечно, функция также зависит от параметров модели RWKV, но поскольку мы используем обученную модель (загруженную отсюда), мы рассматриваем эти параметры как фиксированные. Чтобы преобразовать текст в состояние, мы просто инициализируем состояние нулями и пропускаем лексемы через функцию RWKV одну за другой.

In [2]:
state = np.zeros((N_LAYER, 4, N_EMBD), dtype=np.float32)
for token in tokenizer.encode(context).ids:
    probs, state = RWKV(weights, token, state)

Теперь переменная state содержит представление состояния нашего входного текста, а переменная "probs" - распределение вероятностей, которое модель предсказывает для следующего токена.

Теперь мы можем просто сэмплировать распределение вероятностей (на практике мы избегаем низковероятных токенов в sample_probs()) и добавить в текст еще один токен. Затем мы подаем новую лексему в RWKV и повторяем.

In [3]:
for i in range(100):
    token = sample_probs(probs)
    print(tokenizer.decode([token]), end="", flush=True)
    probs, state = RWKV(weights, token, state)  



Image: Photo by Frantisek Pavlov.

If you want to learn more about dragon cultures in Asia, here are some facts to give you an idea:

• The largest, longest-lived cave dragons inhabit the Himalayan region. These predators have been known to strike when there’s snow.

• Each dragon has a family of siblings that helps it survive during the winter.

• Each dragon is strong enough to hold its own without relying on

Типичным, порожденным продолжением является:

“They’re just like us. They use Tibetan for communication, and for a different reason – they use a language that they’re afraid to use. To protect their secret, they prefer to speak a different language to the local public.”

Конечно, более крупные модели будут работать лучше, чем этот относительно небольшой 430-метровый RWKV.

### Числовые проблемы
На практике возникают некоторые проблемы, которые я проигнорировал в своем упрощенном коде. Самое главное, что на практике мы очень заботимся о производительности/времени выполнения кода. Это приводит к тому, что мы запускаем RWKV параллельно на GPU, используем специализированный GPU-код, написанный на CUDA, используем 16-битные числа с плавающей запятой и многое другое.

Наибольшее число, которое может представлять 16-битное число с плавающей точкой (float16), - 65 504, все, что больше этого, переполняется, что плохо. Большая часть кода не имеет проблем с этим, отчасти потому, что нормализации слоев удерживают значения в разумном диапазоне. Однако внимание RWKV содержит экспоненциально большие числа (exp(bonus + k)). На практике внимание RWKV реализуется таким образом, что мы вычитаем экспоненциальный коэффициент из num и den, чтобы все оставалось в диапазоне float16. Посмотрите, например, функцию time_mixing в RWKV в 150 строках.

### GPT-режим

Мой упрощенный код обрабатывает токены по одному, что гораздо медленнее, чем параллельная обработка, особенно при работе на GPU. Для генерации это невозможно обойти, поскольку нам нужно сделать выборку лексем, прежде чем использовать ее для вычисления следующей. Однако для обучения весь текст уже доступен. Это позволяет нам распараллелить работу с лексемами. Большую часть кода довольно просто распараллелить подобным образом, поскольку зависимость от времени невелика. Например, все дорогостоящие матричные умножения работают на каждом токене независимо, что приводит к хорошей производительности.

Однако внимание RWKV по своей сути является последовательным. К счастью, в нем очень мало вычислений (в 1024 раз меньше, чем в матричных умножениях), поэтому он должен быть быстрым. К сожалению, pytorch не очень хорошо справляется с этой последовательной задачей, поэтому часть внимания становится медленной (даже по сравнению с матричными умножениями). Поэтому я написал оптимизированные ядра CUDA для вычисления внимания RWKV, что и стало моим основным вкладом в проект RWKV.

В JAX есть jax.lax.scan и jax.lax.associative_scan, что позволяет чистой реализации JAX работает лучше, чем чистый pytorch. Тем не менее, по моим оценкам, JAX приведет к замедлению обучения примерно на 40 % по сравнению с CUDA (эта оценка может быть устаревшей, так как она была сделана для обучения относительно небольшой модели объемом 1,5B).

### RWKV: результаты

![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/rwkv_5.png)

### RetNet
- Статья: Retentive Network: A Successor to Transformer for Large Language Models
- Arxiv: 2307.08621
- Организация: Microsoft
- Основная идея: наполовину трансформер, наполовину РНС (где-то мы это уже слышали)
- Репозиторий: Jamie-Stirling/RetNet
- Особенности:
- Матрицы B и C из LRU теперь зависят от входных эмбеддингов, из этого выводится xPos, тип позиционных эмбеддингов
- В отличие от RWKV, не отказываемся от основной идеи транcформеров
- Предупреждение: я использую нотации из LRU из S4, в статье другие буковки

### RetNet: от ЛРНС к xPos
- $E$ — матрица входных эмбеддингов
- $U$ = $V$ в трансформерах = $EW_U$
- $u_j$ = $E_jw_u$ : проекция в скаляр
- $C$ = $Q$ в трансформерах = $EW_C$
- $B$ = $K$ в трансформерах = $EW_B$
- $C_k$ , $B_k$ проекции на k-ой позиции
- $B$ и $C$ зависят от входных эмбеддингов и собирают контекст
- $u_k$ и $y_k$ скаляры, $x_k$ — вектор

$$x_k = Ax_{k-1} + B_ku_k$$

$$y_k = C_kx_k = \sum_{j=0}^{k-1} C_kA^jB^T_k u_{k-j} = \sum_{j=1}^{k} C_kA^{k-j}B^T_k u_j$$

$$A = P(\nu e^{i\theta})P^{-1}$$

$$y_k = \sum_{j=1}^{k} C_k(\nu e^{i\theta})^{k-j}B^T_k u_j$$

$$y_k = \sum_{j=1}^{k} \nu^{k-j}(C_ke^{ik\theta})(B_ke^{ij\theta})^Tu_j$$

### RetNet: от внимания к удержанию
- Теперь в матричной форме
- D одновременно определяет причинную маску и экспоненциальное затухание, аналог HiPPO
- Очень похоже на внимание
- Но без софтмакса и с рекуррентной формой

$$\Theta_k = e^{ik\theta}$$

$$Q = (E W_c) \circ \Theta$$

$$K = (E W_b) \circ \bar{\Theta} \quad \text{Сопряженная матрица}$$

$$V = E W_v$$

$$D_{kj} = \begin{cases}
  v^{k-j}, & k \geq j \\
  0, & k < j 
\end{cases} .$$

$$Y = (Q K^T \circ D) V$$

![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/retnet_4.png)

### RetNet: gated multi-scale
- 𝛾 = 𝜈
- Разные затухания в разных головах
- Нормализация для каждой головы в отдельности
- Гейт как в RWKV
- Глобально архитектура как у трансформеров

![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/retnet_1.png)
![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/retnet_2.png)

### RetNet: Результаты
![](https://raw.githubusercontent.com/sswt/dive2gai/main/unit13/data_z/retnet_3.png)

### Недостатки и проблемы
- Концептуально мы не можем возвращаться к предыдущим токенам, приходится сжимать всю историю в один вектор
- Не было нормальной интеграции в HF Transformers до недавнего времени
- Комплексные числа на GPU
- БПФ на GPU
- Все наработки по квантизации опираются на устройство трансформеров
- Все наработки по объяснимости опираются на устройство трансформеров